## Exercise 3 - Investigating the data


In [1]:
from cifar_helper import CIFAR # note we renamed the helper file so it doesn't syntax error


In [7]:
%matplotlib notebook
from matplotlib import pyplot as plt
